In [1]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np

%load_ext autoreload
%autoreload 2


In [2]:
df_stu, meta_stu = pyreadstat.read_sav('data/CY08MSP_STU_QQQ_HUN.sav')

In [3]:
import os
with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [4]:
df_stu['STUD_MATH'] = 0
cols = utils.meta2docs(meta_stu, excluded=['test','CNTRYID'])

cols.append( Document(
                    page_content= 'Student\'s score in Mathematics',
                    metadata={"year": 2022, "original_col_name": 'STUD_MATH'},
                ))

cols.append( Document(
                    page_content= 'Student\'s score in Reading',
                    metadata={"year": 2022, "original_col_name": 'STUD_READ'},
                ))

cols.append( Document(
                    page_content= 'Student\'s score in Science',
                    metadata={"year": 2022, "original_col_name": 'STUD_SCIE'},
                ))
#drop empty columns
df_stu = df_stu.dropna(axis=1, how='all')

cols2 = []
for col in cols:
    if col.metadata['original_col_name'] in df_stu.columns:
        cols2.append(col)
cols = cols2

cols_vectorstore = FAISS.from_documents(cols, OpenAIEmbeddings())

cols_retriever = cols_vectorstore.as_retriever()

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache


llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


In [7]:
qs = utils.load_questions('questions_c108_20240901_193906.txt')

In [ ]:
res = utils.execute_tests(llm, qs, df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'MATHMOT', 'MATHPREF', 'ST296Q01JA']
content="def run(data):\n    data = data[['ESCS', 'SOCONPA', 'STUD_MATH']].dropna()\n    \n    X = data[['ESCS', 'SOCONPA']]\n    y = data['STUD_MATH']\n    \n    model = sm.OLS(y, sm.add_constant(X)).fit()\n    \n    return model.params, 'Linear Regression'" response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 330, 'total_tokens': 404}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a47b58fc-a0fe-4c6a-a42a-23dde302b4b6-0' usage_metadata={'input_tokens': 330, 'output_tokens': 74, 'total_tokens': 404}
def run(data):
    data = data[['ESCS', 'SOCONPA', 'STUD_MATH']].dropna()
    
    X = data[['ESCS', 'SOCONPA']]
    y = data['STUD_MATH']
    
    model = sm.OLS(y, sm.add_constant(X)).fit()
    
    return model.params, 'Linear Regression'
['ES